AGNEWS RNN 모델 생성, 학습
- [1] 데이터 전처리
    - (한글 제외 모두 제거, 줄임말 한 가지로 처리) ~ skip
    - 토큰화
    - 토큰 데이터 정수 인코딩, 데이터 길이 표준화(패딩)
- [2] 학습 준비
    - Dataset, DataLoader 준비
    - 모델 클래스
    - 학습, 테스트 함수
    - 학습 관련 변수 => DEVICE, OPTIMIZER, MODEL 인스턴스, EPOCHS, BATCH_SIZE, LOSS_FN

*미완성 코드*

In [1]:
import torchdata
import torch
from torchtext.datasets import AG_NEWS

In [2]:
cnt = 0
for a in AG_NEWS(split='train'):
    cnt += 1
    print(a)    # <class 'tuple'> ~ 길이는 2
    if cnt == 5: break

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")
(3, 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.')
(3, "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.")
(3, 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.')
(3, 'Oil prices soar to all-time record, posing new menace t

In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

### 토크나이저 생성
tokenizer = get_tokenizer('basic_english')

### 뉴스 학습 데이터 추출
train_iter = AG_NEWS(split='train')

### 뉴스 테스트 데이터 추출
test_iter = AG_NEWS(split='test')

# DataPipe 타입 >>> generator 타입 형변환
# train_iter = iter(AG_NEWS(split='train'))

In [ ]:
### 한국어라면
# from konlpy.tag import Okt
# tokenizer = Okt()
# 그리고, yield_tokens에서 tokenizer.morphs(text) 사용

In [4]:
### 토큰 제너레이터 함수 : 데이터 추출하여 토큰화
def yield_tokens(data_iter):
    for _, text in data_iter:
        # 라벨, 텍스트 -> 텍스트 토큰화
        yield tokenizer(text)

In [29]:
### 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(train_iter),
                                  specials=['<PAD>', '<UNK>'])

### <UNK> 인덱스 0으로 설정
vocab.set_default_index(vocab['<UNK>'])

In [32]:
vocab(['i', 'want', 'to', 'go', 'home', '<PAD>', '<UNK>'])

[283, 916, 5, 364, 161, 0, 1]

In [7]:
### 텍스트 >>> 정수 인코딩
text_pipeline = lambda x: vocab(tokenizer(x))

text_pipeline('I want to go home')

[282, 915, 4, 363, 160]

In [8]:
### 레이블 >>> 정수 인코딩
### 1 ~ 4 를 0 ~ 3 으로 바꾸려고
label_pipeline = lambda x: int(x) - 1

데이터 가공
- 토큰 데이터 정수 인코딩
- 데이터 길이 표준화 => 데이터의 길이 맞추기 (1개 문장 구성하는 단어 수 통일)

In [23]:
print(type(vocab))

<class 'torchtext.vocab.vocab.Vocab'>


In [33]:
print(len(vocab))

95812


In [45]:
token_to_id = vocab.get_stoi()  # dict (인코딩)
id_to_token = vocab.get_itos()  # list (디코딩)

In [47]:
id_to_token[:10]

['<PAD>', '<UNK>', '.', 'the', ',', 'to', 'a', 'of', 'in', 'and']

In [51]:
PAD_ID = token_to_id["<PAD>"]   # 0
UNK_ID = token_to_id["<UNK>"]   # 1

In [52]:
### 패딩 처리 함수
def pad_sequence(sentences, max_length, pad, start='R'):
    result = []
    for sen in sentences:
        sen = sen[:max_length] if start == 'R' else sen[-1 * max_length:]
        pad_length = max_length - len(sen)  # sen이 원래 max_length 보다 짧았다면 1이상의 값 저장
        padd_sen = sen + [pad]*pad_length if start == 'R' else [pad]*pad_length + sen
        result.append(padd_sen)
    return result

In [ ]:
# 함수 사용하여 일정한 길이의 정수 인코딩 데이터 생성
MAX_LENGTH = 32


데이터 배치(batch)와 반복자 생성
- torch.utils.data.DataLoader : getitem(), len() 구현한 맵 형태(map-style)
- collate_fn() : DataLoader로부터 생성된 샘플 배치 함수
    - 입력 : DataLoader에 배치 크기(batch size)가 있는 배치(batch) 데이터


In [9]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

### 배치크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]    # offset : 파일 포인터

    # 뉴스기사, 라벨을 1개씩 추출해서 저장
    for (_label, _text) in batch:
        # 라벨 인코딩 후 저장
        label_list.append(label_pipeline(_label))

        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)

        # 텍스트 offset 즉, 텍스트 크기/길이 저장
        offsets.append(processed_text.size(0))
    
    # 텐서화 진행
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

In [18]:
### collate_batch 함수 이해하기 위한 테스트 코드
"""cnt = 0
test_list = []

# AG_NEWS 데이터셋 5개 데이터만 추출, 저장
for x in AG_NEWS(split='train'):
    if cnt == 5: break
    # print(x)
    test_list.append(x)
    cnt += 1

label_list, text_list, offsets = collate_batch(test_list)

print(f'label_list : {label_list}')
print(f'text_list : {text_list}')
print(f'text_list.shape : {text_list.shape}')
print(f'offsets : {offsets}')"""

label_list : tensor([2, 2, 2, 2, 2])
text_list : tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
           14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
            8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
          294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
            1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
           27, 

In [12]:
### 데이터로더 생성
train_iter = AG_NEWS(split='train')
trainDL = DataLoader(train_iter,
                     batch_size=16,
                     shuffle=True,
                     collate_fn=collate_batch)  # collate_fn 역할? 최종 데이터 형식?
testDL = DataLoader(test_iter,
                    batch_size=16,
                    shuffle=True,
                    collate_fn=collate_batch)

In [16]:
### 분류 클래스 수와 단어사전 개수
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)

print(f'num_class : {num_class}     vocab_size : {vocab_size}')

num_class : 4     vocab_size : 95811


학습 준비

In [18]:
import torch
from torch import nn

In [19]:
class SentenceClassifier(nn.Module):
    
    def __init__(
            self,
            vocab_size,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout=0.5,
            bidirectional=True,
            model_type='lstm'
    ):
        super().__init__()

        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=0
        )

        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        
        self.classifier = nn.Linear(hidden_dim * 2, 1) if bidirectional else nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

In [ ]:
### DEVICE, CLASSIFIER, CRITERION, OPTIMIZER
from torch import optim

vocab